# Zoningtaxlots QAQC

### Objectives:
+ Combine `qc_versioncomparison.csv` with `qc_versioncomparisonnownullcount.csv`; sort by field name. (Resulting report will show __if the value changed to a different value or to/from a null value__).
+ Add a __difference__ column to `qc_frequencychanges.csv`; sort by field name.
+ Add two fields to the BBL diff report
    + Flag indicating that __lot intersects with a rezoning done since the last version__
    + Flag indicating that __the area of the lot (taken from DTM) has changed by more than +/- 10% since the last version__
+ Rename fields in BBL diff report for the fields showing the new data, using similar naming convention as used for previous data set, e.g., ZD1NEW.

In [2]:
import geopandas as gpd
import pandas as pd
import os
from sqlalchemy import create_engine
from pathlib import Path
import time
from shapely.wkb import dumps, loads
from shapely.wkt import loads as wkt_loads 

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 50)
print(time.strftime("%m/%d/%Y %H:%M:%S"))

12/24/2019 16:30:21


In [3]:
%load_ext dotenv
%dotenv ../.env

In [4]:
conn = create_engine(os.getenv('BUILD_ENGINE'))

In [5]:
recipe_conn = create_engine(os.getenv('RECIPE_ENGINE'))

In [6]:
# Reports the number of records that experienced a change in the value
query = '''
select field, count as diff_count, 
percent as diff_percent,  
        newnullcount, oldnullcount, 
        countnew as total_count_new, 
        countold as total_count_old, 
        (countnew-countold) as total_count_diff 
from(
    SELECT * FROM
        ztl_qc_versioncomparisoncount a 
    JOIN 
        ztl_qc_versioncomparisonnownullcount b
    USING (field)) c
JOIN
frequencychanges d
USING (field)
ORDER BY field;
'''
df_versioncomparison = pd.read_sql(sql=query, con=conn)
df_versioncomparison

,field,diff_count,diff_percent,newnullcount,oldnullcount,total_count_new,total_count_old,total_count_diff
0,commercialoverlay1,52.0,0.0,14,14,74952,74971,-19
1,commercialoverlay2,0.0,0.0,0,0,165,165,0
2,limitedheightdistrict,0.0,0.0,0,0,3037,3037,0
3,specialdistrict1,1.0,0.0,0,0,101895,101895,0
4,specialdistrict2,0.0,0.0,0,0,80,80,0
5,specialdistrict3,0.0,0.0,0,0,0,0,0
6,zoningdistrict1,13.0,0.0,0,0,858314,858349,-35
7,zoningdistrict2,52.0,0.0,38,38,19844,19868,-24
8,zoningdistrict3,1.0,0.0,0,0,207,206,1
9,zoningdistrict4,0.0,0.0,0,0,13,13,0


In [7]:
# Reports the full zoning comarison table
query = '''
SELECT bblnew, bblprev, 
        zd1new, zd1prev, zd2new, zd2prev, zd3new, zd3prev, zd4new, zd4prev, 
        zmcnew, zmcprev, zmnnew, zmnprev, 
        co1new, co1prev, co2new, co2prev, 
        sd1new, sd1prev, sd2new, sd2prev, sd3new, sd3prev, 
        lhdnew, lhdprev, 
        inzonechange, mihflag, mihoption, 
        geom from bbldiffs;
'''
bbldiffs = gpd.GeoDataFrame.from_postgis(sql=query, con=conn)
bbldiffs

,bblnew,bblprev,zd1new,zd1prev,zd2new,zd2prev,zd3new,zd3prev,zd4new,zd4prev,zmcnew,zmcprev,zmnnew,zmnprev,co1new,co1prev,co2new,co2prev,sd1new,sd1prev,sd2new,sd2prev,sd3new,sd3prev,lhdnew,lhdprev,inzonechange,mihflag,mihoption,geom
0,4019050006,4019050006,R6A,R6A,None,None,None,None,None,None,None,None,14A,14A,C1-2,C2-2,None,None,None,None,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((-73.86373 40.73758, -73.86408 ..."
1,4019040006,4019040006,R6A,R6A,R5,R5,None,None,None,None,None,None,14A,14A,C1-2,C2-2,None,None,None,None,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((-73.86464 40.73740, -73.86462 ..."
2,3057140009,3057140009,C4-2,R6,None,None,None,None,None,None,None,None,22A,22A,None,C1-3,None,None,None,None,None,None,None,None,None,None,Y,None,None,"MULTIPOLYGON (((-74.00935 40.63516, -74.00939 ..."
3,4007040042,4007040042,R7X,M1-1,None,None,None,None,None,None,None,None,9B,9B,C2-4,None,None,None,None,None,None,None,None,None,None,None,Y,None,None,"MULTIPOLYGON (((-73.91787 40.75397, -73.91808 ..."
4,2039850005,2039850005,R6,R5A,R5A,R6,None,None,None,None,None,None,4B,4B,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((-73.84652 40.84029, -73.84645 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,4019030056,4019030056,R6A,R6A,None,None,None,None,None,None,None,None,14A,14A,C1-2,C2-2,None,None,None,None,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((-73.86541 40.73687, -73.86545 ..."
115,4019050047,4019050047,R6A,R6A,None,None,None,None,None,None,None,None,14A,14A,C1-2,C2-2,None,None,None,None,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((-73.86373 40.73758, -73.86339 ..."
116,4006210032,4006210032,R5,C4-2A,C4-2A,R5,None,None,None,None,None,None,9A,9A,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((-73.92400 40.76080, -73.92395 ..."
117,4019060001,4019060001,R6A,R6A,R6B,R6B,None,None,None,None,Y,Y,10B,10B,C1-2,C2-2,None,None,None,None,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((-73.86295 40.73809, -73.86263 ..."


## DTM Comparison

In [8]:
version_old = '2019/10/02'
version_new = '2019/11/04'

In [9]:
# Reports lots that had an area change
query = f'''
with dtm_compare as (
    SELECT bbl, geom_new, geom_old, (case when geom_new = geom_old then 0 else 1 end) flag 
    FROM 
    (SELECT bbl, ST_Multi(ST_Union(f.wkb_geometry)) as geom_new 
        FROM dof_dtm."{version_new}" f GROUP BY bbl ) a
    JOIN 
    (SELECT bbl, ST_Multi(ST_Union(f.wkb_geometry)) as geom_old 
        FROM dof_dtm."{version_old}" f GROUP BY bbl ) b
    USING(bbl))
, changed as (
    SELECT *, (st_area(geom_new)-st_area(geom_old))/st_area(geom_old) as area_diff 
    FROM dtm_compare
    WHERE flag = 1)
SELECT * FROM changed WHERE area_diff > 0.1 OR area_diff < -0.1;
'''

In [10]:
bbl_areachange = gpd.GeoDataFrame.from_postgis(sql=query, con=recipe_conn, geom_col='geom_new')

In [11]:
bbl_areachange_new = bbl_areachange[['bbl', 'geom_new', 'area_diff']]

In [12]:
bbl_areachange_old = bbl_areachange[['bbl', 'geom_old', 'area_diff']]
bbl_areachange_old.loc[:, 'geom_old'] = bbl_areachange_old['geom_old'].apply(lambda x: wkt_loads(loads(x,  hex=True).wkt))
bbl_areachange_old=gpd.GeoDataFrame(bbl_areachange_old, geometry='geom_old')

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [13]:
from ipyleaflet import Map, basemaps, GeoData, basemap_to_tiles, LayersControl, FullScreenControl, Popup, Marker
from ipywidgets import HTML

m = Map(center=(40.730610, -73.935242), zoom=11)

dark_matter_layer = basemap_to_tiles(basemaps.CartoDB.DarkMatter, close_popup_on_click=True)
m.add_layer(dark_matter_layer)

In [14]:
new = GeoData(geo_dataframe = bbl_areachange_new, 
              style={'color': 'green', 'opacity':10, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.3},
              name = 'new')

old = GeoData(geo_dataframe = bbl_areachange_old, 
              style={'color': 'red', 'opacity':10, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.3}, 
              name = 'old')

In [15]:
m.add_layer(new)
m.add_layer(old)
m.add_control(LayersControl())
m.add_control(FullScreenControl())

In [16]:
for i in range(bbl_areachange_new.shape[0]):
    center = (bbl_areachange_new.loc[i, 'geom_new'].centroid.y,
              bbl_areachange_new.loc[i, 'geom_new'].centroid.x)
    bbl = str(list(bbl_areachange_new['bbl'])[i])
    area_change = round(list(bbl_areachange_new['area_diff'])[i]*100, 2)
    marker = Marker(location=center)
    m.add_layer(marker)
    marker.popup = HTML(value=f'''<a href=https://zola.planning.nyc.gov/bbl/{bbl}> {bbl} </a>
                                  <p> area change: {area_change}% </p>''')

In [17]:
m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …